In [0]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, mean_squared_error
import sklearn
from datetime import datetime
import lightgbm as lgb

In [2]:
skater_feature_set = pd.read_csv('feature_matrix_w_position.csv')

goalie_feature_set = pd.read_csv('goalie_feature_matrix.csv').sort_values('season')
goalie_feature_set.tail(10)


,Unnamed: 0,player_id,game_id,season,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,goals_against_5,goals_against_10,goals_against_20,goals_against_season,save_percent_1,save_percent_5,save_percent_10,save_percent_20,save_percent_season,shots_against_1,shots_against_5,shots_against_10,shots_against_20,shots_against_season,wins_1,wins_5,wins_10,wins_20,wins_season,shutouts_1,shutouts_5,shutouts_10,shutouts_20,shutouts_season,ot_loss_1,ot_loss_5,ot_loss_10,ot_loss_20,ot_loss_season,es_shots_1,es_shots_5,es_shots_10,es_shots_20,es_shots_season,es_save_percent_1,es_save_percent_5,es_save_percent_10,es_save_percent_20,es_save_percent_season,pp_shots_1,pp_shots_5,pp_shots_10,pp_shots_20,pp_shots_season,pp_save_percent_1,pp_save_percent_5,pp_save_percent_10,pp_save_percent_20,pp_save_percent_season,sh_shots_1,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season,dfs_points
8288,8288,8475622,540,2019,A,24.0,27.2,27.6,28.35,27.800000,6.0,4.4,3.9,3.30,3.066667,80.000000,85.210253,86.852962,89.149242,89.351102,30.0,31.6,31.5,31.65,30.866667,0.0,0.0,0.1,0.35,0.400000,0.0,0.0,0.0,0.10,0.133333,1.0,0.6,0.4,0.20,0.266667,27.0,25.6,25.8,26.20,25.066667,85.185185,88.494168,88.289356,89.867578,90.519702,2.0,4.8,4.8,4.55,4.933333,50.0,69.500000,81.416667,83.813492,85.825397,1.0,1.2,0.9,0.90,0.866667,0.0,50.000000,71.428571,80.769231,75.000000,-4.9
8289,8289,8475622,692,2019,H,23.0,27.0,27.4,27.65,27.500000,6.0,4.6,4.2,3.30,3.250000,79.310345,84.520598,85.855425,88.812434,88.723555,29.0,31.6,31.6,30.95,30.750000,0.0,0.0,0.0,0.35,0.375000,0.0,0.0,0.0,0.10,0.125000,0.0,0.4,0.4,0.20,0.250000,23.0,25.6,25.6,25.30,24.937500,86.956522,88.494168,88.185008,89.825160,90.297003,5.0,4.8,5.0,4.75,4.937500,40.0,65.500000,75.416667,85.813492,82.961310,1.0,1.2,1.0,0.90,0.875000,100.0,50.000000,75.000000,80.769231,77.272727,2.8
8290,8290,8475622,774,2019,H,29.0,28.2,28.5,27.75,27.588235,5.0,4.8,4.1,3.35,3.352941,85.294118,84.542384,86.884837,88.722301,88.521823,34.0,33.0,32.6,31.10,30.941176,0.0,0.0,0.0,0.35,0.352941,0.0,0.0,0.0,0.10,0.117647,0.0,0.4,0.4,0.20,0.235294,30.0,28.2,26.5,25.75,25.235294,83.333333,88.690246,89.375485,89.706112,89.887375,4.0,4.0,5.2,4.50,4.882353,100.0,68.000000,75.416667,86.369048,83.963585,0.0,0.8,0.9,0.85,0.823529,NaN,33.333333,71.428571,79.166667,77.272727,2.8
8291,8291,8475622,782,2019,H,4.0,24.2,24.7,26.30,26.277778,0.0,3.8,3.9,3.10,3.166667,100.000000,87.990660,87.339382,89.380195,89.159499,4.0,28.0,28.6,29.40,29.444444,0.0,0.0,0.0,0.35,0.333333,0.0,0.0,0.0,0.10,0.111111,0.0,0.4,0.3,0.20,0.222222,1.0,24.2,22.8,24.05,23.888889,100.000000,90.595008,89.638642,90.420398,90.449188,3.0,3.4,4.9,4.50,4.777778,100.0,71.333333,77.083333,86.369048,84.854497,0.0,0.4,0.9,0.85,0.777778,NaN,50.000000,71.428571,79.166667,77.272727,11.9
8292,8292,8475622,792,2019,H,27.0,21.4,25.4,26.05,26.315789,2.0,3.8,4.0,3.05,3.105263,93.103448,87.541582,87.125917,89.463939,89.367076,29.0,25.2,29.4,29.10,29.421053,0.0,0.0,0.0,0.35,0.315789,0.0,0.0,0.0,0.10,0.105263,0.0,0.2,0.3,0.20,0.210526,29.0,22.0,23.8,23.95,24.157895,93.103448,89.715698,89.475303,90.559441,90.588886,0.0,2.8,4.8,4.45,4.526316,NaN,72.500000,74.537037,85.651629,84.854497,0.0,0.4,0.8,0.70,0.736842,NaN,50.000000,66.666667,77.272727,77.272727,17.9
8293,8293,8475622,813,2019,H,22.0,21.0,24.1,26.10,26.100000,1.0,2.8,3.6,3.00,3.000000,95.652174,90.672017,87.941135,89.681331,89.681331,23.0,23.8,27.7,29.10,29.100000,1.0,0.2,0.1,0.35,0.350000,0.0,0.0,0.0,0.10,0.100000,0.0,0.0,0.3,0.20,0.200000,18.0,20.2,22.9,23.85,23.850000,94.444444,91.567550,90.030859,90.781664,90.781664,5.0,3.4,4.1,4.55,4.550000,100.0,85.000000,76.388889,85.651629,85.651629,0.0,0.2,0.7,0.70,0.700000,NaN,100.000000,60.000000,77.272727,77.272727,-11.9
8294,8294,8475622,840,2019,A,8.0,18.0,22.5,25.10,25.238095,5.0,2.6,3.6,3.20,3.095238,61.538462,87.117640,85.819119,87.930668,88.341194,13.0,20.6,26.1,28.30,28.333333

In [3]:
## Perform lgb analysis on 3 sets of players (forwards, defensemen, goalies) ##

skater_feature_set = skater_feature_set.drop(columns = ['Unnamed: 0'], axis = 1)

skater_feature_set = skater_feature_set.replace('H', 1)
skater_feature_set = skater_feature_set.replace('A', 0)

skater_feature_set = skater_feature_set.dropna()
print(len(skater_feature_set))

155610


In [4]:
after_november_1_skaters = skater_feature_set[(skater_feature_set['season'] == 2019) & (skater_feature_set['game_id'] >= 196)]
november_1_skaters = after_november_1_skaters[(skater_feature_set['game_id'] <= 203)]


skater_feature_set.drop(after_november_1_skaters.index, inplace = True)
print(len(november_1_skaters))
november_1_skaters

254


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,player_id,game_id,season,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,assists_5,assists_10,assists_20,assists_season,shots_1,shots_5,shots_10,shots_20,shots_season,hits_1,hits_5,hits_10,hits_20,hits_season,blocks_1,blocks_5,blocks_10,blocks_20,blocks_season,es_toi_1,es_toi_5,es_toi_10,es_toi_20,es_toi_season,pp_toi_1,pp_toi_5,pp_toi_10,pp_toi_20,pp_toi_season,pp_goals_1,pp_goals_5,pp_goals_10,pp_goals_20,pp_goals_season,pp_assists_1,pp_assists_5,pp_assists_10,pp_assists_20,pp_assists_season,takeaways_1,takeaways_5,takeaways_10,takeaways_20,takeaways_season,giveaways_1,giveaways_5,giveaways_10,giveaways_20,giveaways_season,plus_minus_1,plus_minus_5,plus_minus_10,plus_minus_20,plus_minus_season,sh_goals_1,sh_goals_5,sh_goals_10,sh_goals_20,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points,primaryPositioncode,primaryPositionname,primaryPositiontype
1732,8466138,203,2019,1,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.3,0.50,0.307692,0.0,0.6,0.9,0.95,0.846154,1.0,0.4,0.2,0.20,0.307692,0.0,0.4,0.2,0.25,0.230769,787.0,754.2,751.5,754.30,739.384615,57.0,106.8,88.4,138.80,107.846154,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.1,0.25,0.076923,1.0,0.8,0.9,0.90,0.923077,0.0,0.8,0.5,0.55,0.615385,0.0,0.0,-0.2,-0.40,-0.461538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,7.3,C,Center,Forward
2124,8466139,203,2019,1,0.0,0.0,0.2,0.15,0.222222,0.0,0.6,0.4,0.20,0.444444,0.0,0.8,1.7,1.75,1.333333,3.0,1.2,1.4,1.15,1.555556,0.0,0.4,0.5,0.45,0.555556,814.0,879.6,872.5,825.00,855.888889,30.0,98.4,82.2,88.45,86.333333,0.0,0.0,0.1,0.05,0.111111,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.3,0.35,0.222222,1.0,0.2,0.1,0.25,0.111111,0.0,0.4,0.4,0.05,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,11.8,C,Center,Forward
14045,8470047,199,2019,0,0.0,0.2,0.1,0.15,0.076923,1.0,0.2,0.2,0.20,0.230769,1.0,2.0,1.3,1.30,1.076923,1.0,0.4,0.4,0.25,0.307692,0.0,0.4,0.4,0.30,0.307692,887.0,804.2,770.8,718.45,773.769231,82.0,167.6,113.2,109.15,130.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.15,0.000000,0.0,1.0,0.8,0.70,0.846154,1.0,-0.2,-0.4,-0.25,-0.461538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,6.5,C,Center,Forward
14636,8470110,199,2019,0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.10,0.000000,1.0,0.8,0.9,0.90,0.888889,0.0,0.2,0.1,0.35,0.111111,1.0,1.2,1.0,0.85,1.111111,955.0,889.6,824.8,888.25,804.888889,0.0,1.0,0.5,0.70,0.555556,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.2,0.20,0.222222,1.0,0.4,0.5,0.75,0.555556,-1.0,-0.6,0.0,-0.15,-0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,3.0,D,Defenseman,Defenseman
15032,8470144,199,2019,0,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.0,0.10,0.000000,1.0,1.6,1.4,1.50,1.400000,1.0,1.0,1.0,1.20,1.000000,2.0,0.6,0.5,0.50,0.500000,700.0,646.4,640.5,694.20,640.500000,11.0,15.0,54.4,77.65,54.400000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.05,0.000000,0.0,0.2,0.2,0.35,0.200000,0.0,0.0,0.1,0.20,0.100000,1.0,0.0,-0.2,-0.25,-0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,4.5,C,Center,Forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213254,8480039,199,2019,1,0.0,0.0,0.1,0.15,0.166667,0.0,0.2,0.4,0.25,0.333333,1.0,0.6,0.9,0.95,1.083333,1.0,0.2,0.2,0.35,0.333333,0.0,0.8,0.5,0.35,0.416667,761.0,700.4,707.5,615.05,707.083333,57.0,90.8,100.7,110.15,116.250000,0.0,0.0,0.0,0.05,0.083333,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.2,0.35,0.250000,0.0,0.4,0.7,0.90,0.833333,0.0,-0.6,0.0,-0.05,-0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,14.5,C,Center,Forward
214255,8480145,203,2019,0,0.0,0.0,0.1,0.10,0.153846,0.0,0.6,0.5,0.45,0.461538,2.0,1.6,1.7,1.45,1.769231,3.0,2.6,2.3,1.95,2.230769,0.0,1.6,1.3,1.15,1.307692,1104.0,1039.2,1070.3,1038.40,1102.692308,102.0,235.0,185.8,114.45,174.923077,0.0,0.

In [0]:
goalie_feature_set = goalie_feature_set.drop(columns = ['Unnamed: 0'], axis = 1)

goalie_feature_set = goalie_feature_set.replace('H', 1)
goalie_feature_set = goalie_feature_set.replace('A', 0)

In [6]:
# For the goalie feature matrix, we don't want to drop the nulls, as we don't have that much data
# We will take the average for that player that season for any columns that are null

goalie_feature_set = goalie_feature_set.fillna(goalie_feature_set.groupby(['player_id','season']).transform('mean'))
print(len(goalie_feature_set))
goalie_feature_set.sort_values('season').tail(10)

13003


,player_id,game_id,season,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,goals_against_5,goals_against_10,goals_against_20,goals_against_season,save_percent_1,save_percent_5,save_percent_10,save_percent_20,save_percent_season,shots_against_1,shots_against_5,shots_against_10,shots_against_20,shots_against_season,wins_1,wins_5,wins_10,wins_20,wins_season,shutouts_1,shutouts_5,shutouts_10,shutouts_20,shutouts_season,ot_loss_1,ot_loss_5,ot_loss_10,ot_loss_20,ot_loss_season,es_shots_1,es_shots_5,es_shots_10,es_shots_20,es_shots_season,es_save_percent_1,es_save_percent_5,es_save_percent_10,es_save_percent_20,es_save_percent_season,pp_shots_1,pp_shots_5,pp_shots_10,pp_shots_20,pp_shots_season,pp_save_percent_1,pp_save_percent_5,pp_save_percent_10,pp_save_percent_20,pp_save_percent_season,sh_shots_1,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season,dfs_points
3836,8471469,770,2019,1,30.0,28.0,25.4,23.50,24.137931,1.0,2.4,2.6,2.95,2.724138,96.774194,92.438556,87.683894,86.299068,88.082624,31.0,30.4,28.0,26.45,26.862069,1.0,0.6,0.5,0.45,0.551724,0.0,0.0,0.0,0.05,0.103448,0.0,0.0,0.0,0.05,0.103448,25.0,25.4,23.8,21.65,22.586207,96.000000,94.090110,89.816484,89.315709,90.824604,6.0,4.6,3.9,4.30,3.827586,100.000000,76.476190,71.011905,74.642857,72.757143,0.0,0.4,0.3,0.50,0.448276,81.250000,100.000000,100.000000,75.000000,75.000000,4.2
4035,8471679,86,2019,1,26.0,31.4,29.7,27.65,32.000000,3.0,3.2,2.7,2.60,3.500000,89.655172,90.724479,91.593984,91.255654,90.018501,29.0,34.6,32.4,30.25,35.500000,1.0,0.4,0.5,0.55,0.500000,0.0,0.0,0.0,0.05,0.000000,0.0,0.2,0.2,0.10,0.250000,24.0,29.0,28.1,26.15,28.500000,91.666667,91.487455,92.195193,91.388694,90.972222,3.0,4.4,3.3,3.40,5.500000,66.666667,78.472222,83.981481,88.148148,78.472222,2.0,1.2,1.0,0.70,1.500000,100.000000,100.000000,100.000000,100.000000,100.000000,2.8
3823,8471469,460,2019,1,28.0,20.4,20.9,25.05,23.250000,3.0,3.2,2.8,2.45,2.625000,90.322581,83.267877,86.722323,89.804057,88.616261,31.0,23.6,23.7,27.50,25.875000,0.0,0.2,0.4,0.65,0.562500,0.0,0.2,0.3,0.15,0.187500,1.0,0.2,0.2,0.15,0.187500,22.0,18.4,19.6,22.65,21.875000,86.363636,90.606061,89.995338,92.527485,91.914352,7.0,4.4,3.6,4.35,3.562500,100.000000,65.625000,80.729167,75.324074,71.130952,2.0,0.8,0.5,0.50,0.437500,100.000000,100.000000,100.000000,91.666667,87.500000,12.3
3821,8471469,401,2019,0,12.0,16.6,20.4,25.60,22.357143,5.0,3.8,2.8,2.60,2.785714,70.588235,79.824573,86.653027,89.555017,87.681257,17.0,20.4,23.2,28.20,25.142857,0.0,0.2,0.4,0.60,0.571429,0.0,0.0,0.2,0.10,0.142857,0.0,0.0,0.2,0.15,0.142857,9.0,16.0,19.4,23.75,21.571429,100.000000,87.333333,90.487762,92.417637,91.733285,8.0,4.4,3.7,4.05,3.357143,37.500000,61.458333,78.645833,75.324074,66.319444,0.0,0.0,0.1,0.40,0.214286,81.250000,80.269608,100.000000,90.000000,75.000000,31.7
3559,8471418,873,2019,1,23.0,24.6,27.2,27.15,26.954545,2.0,2.4,2.0,2.00,2.136364,92.000000,89.843828,92.258314,92.559210,92.116499,25.0,27.0,29.2,29.15,29.090909,1.0,0.6,0.7,0.65,0.590909,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.05,0.045455,18.0,23.0,24.5,24.05,23.681818,94.444444,92.313519,94.350267,94.144678,93.828495,7.0,3.8,4.2,4.55,4.863636,85.714286,80.476190,86.238095,82.738095,82.602814,0.0,0.2,0.5,0.55,0.545455,94.444444,100.000000,83.333333,90.000000,91.666667,19.3
3560,8471418,910,2019,0,24.0,23.4,25.9,26.90,26.826087,1.0,2.4,2.0,1.90,2.086957,96.000000,89.688989,92.121472,92.827960,92.285347,25.0,25.8,27.9,28.80,28.913043,1.0,0.6,0.7,0.70,0.608696,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.05,0.043478,21.0,21.4,23.6,23.85,23.565217,95.238095,92.050794,94.207410,94.506583,93.889782,1.0,3.6,3.5,4.25,4.695652,100.000000,80.476190,86.238095,82.738095,83.359213,3.0,0.8,0.8,0.70,0.652174,100.000000,100.000000,87.500000,91.666667,92.857143,20.4
3561,8471418,944,2019,1,42.0,27.0,25.6,28.05,27.458333,4.0,2.8,2.1,2.05,2.166667,91.304348,89.488321,91.876907,92.643177,92.24447

In [10]:
n1_goalies = goalie_feature_set[(goalie_feature_set['season'] == 2019) & (goalie_feature_set['game_id'] >= 197) & (goalie_feature_set['game_id'] <= 203)].sort_values('game_id')
print(n1_goalies)

       player_id  game_id  ...  sh_save_percent_season  dfs_points
3379     8471306      197  ...              100.000000        22.1
10817    8476883      197  ...               80.000000         0.7
11576    8476999      198  ...              100.000000        -1.4
7099     8474651      198  ...              100.000000        22.8
5647     8473541      199  ...               91.666667         4.2
2256     8470657      199  ...              100.000000        -6.3
9564     8475852      199  ...               75.000000        15.8
12361    8478007      200  ...              100.000000         4.1
10218    8476412      200  ...               88.888889        14.4
3542     8471418      201  ...              100.000000        32.1
9215     8475831      201  ...               91.666667        11.9
6492     8474593      202  ...               87.500000        13.9
10481    8476434      202  ...               85.714286        32.8
11454    8476945      203  ...               93.333333        

In [11]:
after_november_1_goalies = goalie_feature_set[(goalie_feature_set['season'] == 2019) & (goalie_feature_set['game_id'] >= 196)]
november_1_goalies = after_november_1_goalies[(after_november_1_goalies['game_id'] <= 203)]

goalie_feature_set.drop(after_november_1_goalies.index, inplace = True)
print(len(november_1_goalies))
after_november_1_goalies.head(10)

17


,player_id,game_id,season,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,goals_against_5,goals_against_10,goals_against_20,goals_against_season,save_percent_1,save_percent_5,save_percent_10,save_percent_20,save_percent_season,shots_against_1,shots_against_5,shots_against_10,shots_against_20,shots_against_season,wins_1,wins_5,wins_10,wins_20,wins_season,shutouts_1,shutouts_5,shutouts_10,shutouts_20,shutouts_season,ot_loss_1,ot_loss_5,ot_loss_10,ot_loss_20,ot_loss_season,es_shots_1,es_shots_5,es_shots_10,es_shots_20,es_shots_season,es_save_percent_1,es_save_percent_5,es_save_percent_10,es_save_percent_20,es_save_percent_season,pp_shots_1,pp_shots_5,pp_shots_10,pp_shots_20,pp_shots_season,pp_save_percent_1,pp_save_percent_5,pp_save_percent_10,pp_save_percent_20,pp_save_percent_season,sh_shots_1,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season,dfs_points
11469,8476945,491,2019,0,1.0,23.8,25.6,27.45,28.320000,1.0,1.4,1.6,1.85,2.040000,50.000000,86.535132,90.682915,91.439561,91.468219,2.0,25.2,27.2,29.30,30.360000,0.0,0.6,0.7,0.60,0.600000,0.0,0.2,0.2,0.15,0.120000,0.0,0.2,0.1,0.10,0.080000,2.0,22.0,23.8,25.05,25.880000,50.000000,87.206298,90.685975,91.590606,92.024732,0.0,2.8,2.5,3.40,3.520000,84.763461,93.750000,96.428571,89.270833,86.349206,0.0,0.4,0.9,0.85,0.960000,99.019608,100.0,100.000000,100.000000,97.619048,6.3
13000,8480925,1065,2019,0,19.0,30.2,25.3,27.55,26.935484,5.0,3.0,2.2,2.40,2.225806,79.166667,89.853482,91.183355,91.448798,92.029113,24.0,33.2,27.5,29.95,29.161290,0.0,0.6,0.6,0.60,0.612903,0.0,0.0,0.1,0.05,0.064516,0.0,0.2,0.2,0.20,0.129032,21.0,26.8,22.7,25.00,24.741935,76.190476,87.936508,89.025674,90.281832,91.524410,2.0,5.2,4.0,3.80,3.516129,100.000000,100.000000,100.000000,92.324561,89.511494,1.0,1.2,0.8,1.15,0.903226,100.000000,100.0,100.000000,100.000000,100.000000,10.9
11457,8476945,264,2019,1,32.0,31.4,30.4,31.45,31.153846,1.0,2.0,2.1,2.40,2.230769,96.969697,92.084398,92.677979,92.388022,92.539937,33.0,33.4,32.5,33.85,33.384615,1.0,0.6,0.5,0.50,0.538462,0.0,0.0,0.1,0.10,0.076923,0.0,0.2,0.1,0.10,0.076923,30.0,27.4,26.9,28.20,27.846154,96.666667,91.684599,94.046322,93.530930,93.551227,2.0,5.0,4.5,4.65,4.461538,100.000000,96.666667,78.333333,80.599415,81.410256,1.0,1.0,1.1,1.00,1.076923,100.000000,100.0,93.333333,96.969697,95.238095,17.2
11458,8476945,276,2019,1,26.0,30.8,30.3,31.10,30.785714,2.0,2.2,2.1,2.50,2.214286,92.857143,91.322493,92.653349,92.030879,92.562595,28.0,33.0,32.4,33.60,33.000000,1.0,0.6,0.5,0.50,0.571429,0.0,0.0,0.1,0.05,0.071429,0.0,0.2,0.1,0.10,0.071429,27.0,28.0,27.2,27.90,27.785714,92.592593,90.203117,93.305582,93.160560,93.482753,0.0,3.8,4.3,4.65,4.142857,84.763461,100.000000,81.481481,80.599415,81.410256,1.0,1.2,0.9,1.05,1.071429,100.000000,100.0,100.000000,97.222222,95.833333,0.7
12999,8480925,1047,2019,0,26.0,30.4,26.8,27.40,27.200000,4.0,2.2,1.9,2.15,2.133333,86.666667,93.067767,92.711132,92.490464,92.472646,30.0,32.6,28.7,29.55,29.333333,0.0,0.8,0.7,0.65,0.633333,0.0,0.0,0.1,0.10,0.066667,1.0,0.2,0.2,0.20,0.133333,28.0,26.4,23.5,24.60,24.866667,85.714286,91.645781,91.406626,91.472308,92.053167,2.0,5.2,4.4,3.85,3.566667,100.000000,100.000000,96.296296,92.324561,89.136905,0.0,1.0,0.8,1.10,0.900000,100.000000,100.0,100.000000,100.000000,100.000000,-4.2
11460,8476945,325,2019,0,31.0,28.2,30.1,30.15,30.187500,3.0,2.2,2.2,2.40,2.375000,91.176471,92.375662,92.113853,91.913440,91.940800,34.0,30.4,32.3,32.55,32.562500,1.0,0.6,0.6,0.55,0.562500,0.0,0.0,0.1,0.05,0.062500,0.0,0.2,0.1,0.10,0.062500,30.0,26.8,26.9,27.40,27.437500,90.000000,92.111111,91.911903,92.881989,92.734909,2.0,2.8,4.4,4.10,4.062500,100.000000,95.000000,94.814815,81.257310,82.555556,2.0,0.8,1.0,1.05,1.062500,100.000000,100.0,100.000000,96.969697,96.296296,32.1
11459,8476945,300,2019,0,21.0,32.2,29.3,30.55,30.133333,4.0,2.0,2.1,2.50,2.333333,84.000000,93.385651,92.196206,91.786435,91.991755,25.0,34.2

In [12]:
november_1_goalies.head(10)
# november_1_skaters.head(10)

,player_id,game_id,season,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,goals_against_5,goals_against_10,goals_against_20,goals_against_season,save_percent_1,save_percent_5,save_percent_10,save_percent_20,save_percent_season,shots_against_1,shots_against_5,shots_against_10,shots_against_20,shots_against_season,wins_1,wins_5,wins_10,wins_20,wins_season,shutouts_1,shutouts_5,shutouts_10,shutouts_20,shutouts_season,ot_loss_1,ot_loss_5,ot_loss_10,ot_loss_20,ot_loss_season,es_shots_1,es_shots_5,es_shots_10,es_shots_20,es_shots_season,es_save_percent_1,es_save_percent_5,es_save_percent_10,es_save_percent_20,es_save_percent_season,pp_shots_1,pp_shots_5,pp_shots_10,pp_shots_20,pp_shots_season,pp_save_percent_1,pp_save_percent_5,pp_save_percent_10,pp_save_percent_20,pp_save_percent_season,sh_shots_1,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season,dfs_points
11454,8476945,203,2019,0,14.0,26.4,29.100000,29.450000,29.100000,5.0,2.2,2.500000,2.350000,2.500000,73.684211,91.006761,91.294807,92.205895,91.294807,19.0,28.6,31.600000,31.80,31.600000,0.0,0.400000,0.500000,0.50,0.500000,0.0,0.200000,0.100000,0.150000,0.100000,0.0,0.000000,0.000000,0.050000,0.000000,17.0,24.000000,26.600000,26.450000,26.600000,70.588235,91.905003,92.833119,93.639192,92.833119,2.0,4.200000,4.100000,4.500000,4.100000,100.000000,74.666667,75.833333,79.521605,75.833333,0.0,0.400000,0.900000,0.850000,0.900000,99.019608,100.000000,93.333333,87.878788,93.333333,37.7
12361,8478007,200,2019,0,19.0,25.7,26.143478,26.780769,24.111111,0.0,2.0,1.834783,1.819231,2.888889,100.000000,91.714166,92.461940,92.807101,90.176305,19.0,27.7,27.978261,28.60,27.000000,0.0,0.428571,0.517391,0.55,0.111111,0.0,0.178571,0.213043,0.223077,0.000000,0.0,0.185714,0.121739,0.107692,0.222222,15.0,25.007143,25.321739,25.903846,22.777778,100.000000,92.107562,92.861819,93.212621,89.921290,3.0,2.328571,2.326087,2.376923,3.444444,100.000000,85.083475,87.112557,88.062615,80.300454,1.0,0.364286,0.330435,0.319231,0.777778,100.000000,100.000000,100.000000,100.000000,100.000000,4.1
12437,8478406,196,2019,1,12.0,22.8,24.000000,25.250000,20.166667,4.0,3.2,2.700000,3.000000,3.000000,75.000000,87.093516,88.595369,88.417193,85.540893,16.0,26.0,26.700000,28.25,23.166667,0.0,0.400000,0.500000,0.40,0.333333,0.0,0.200000,0.100000,0.050000,0.166667,0.0,0.200000,0.100000,0.050000,0.166667,15.0,19.400000,21.300000,23.600000,17.666667,80.000000,90.120370,89.946337,90.037867,88.063272,0.0,6.000000,4.400000,3.900000,5.000000,83.203373,81.666667,84.583333,74.037698,81.666667,1.0,0.600000,1.000000,0.750000,0.500000,0.000000,66.666667,80.000000,88.888889,66.666667,11.1
11576,8476999,198,2019,0,41.0,35.4,30.400000,28.400000,35.400000,0.0,2.6,2.900000,3.050000,2.600000,100.000000,92.709919,90.977283,89.902935,92.709919,41.0,38.0,33.300000,31.45,38.000000,1.0,0.600000,0.400000,0.35,0.600000,1.0,0.200000,0.100000,0.050000,0.200000,0.0,0.200000,0.200000,0.150000,0.200000,34.0,30.400000,27.300000,26.500000,30.400000,100.000000,94.288673,92.796351,91.400280,94.288673,7.0,6.400000,5.100000,4.000000,6.400000,100.000000,85.841270,79.920635,74.890873,85.841270,0.0,1.200000,0.900000,0.950000,1.200000,97.368421,100.000000,100.000000,90.909091,100.000000,-1.4
12704,8479394,196,2019,0,8.0,12.2,18.700000,24.200000,16.285714,1.0,3.0,2.800000,2.800000,2.571429,88.888889,79.253968,85.030671,86.244463,83.798917,9.0,15.2,21.500000,27.00,18.857143,0.0,0.000000,0.200000,0.35,0.285714,0.0,0.000000,0.100000,0.050000,0.142857,0.0,0.200000,0.100000,0.050000,0.142857,9.0,14.200000,18.700000,24.050000,17.000000,88.888889,80.489197,85.553908,85.527316,85.078490,0.0,1.000000,2.100000,2.200000,1.428571,75.023810,62.500000,70.000000,84.117647,58.333333,0.0,0.000000,0.700000,0.750000,0.428571,97.777778,98.253968,100.000000,100.000000,100.000000,11.6
6492,8474593,202,2019,0,21.0,28.8,29.500000,29.300000,28.375000,3.0,2.8,2.600000,2.7500

In [0]:
## splitting the skater feature set into forwards + defensemen ##
skater_feature_set.iloc[0]

defense_feature_set = skater_feature_set.loc[skater_feature_set['primaryPositiontype'] == 'Defenseman']

forward_feature_set = skater_feature_set.loc[skater_feature_set['primaryPositiontype'] == 'Forward']


In [0]:
X_forwards = forward_feature_set.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season', 'primaryPositioncode', 'primaryPositionname', 'primaryPositiontype'], axis = 1)
y_forwards = forward_feature_set['dfs_points'].tolist()

X_defense = defense_feature_set.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season', 'primaryPositioncode', 'primaryPositionname', 'primaryPositiontype'], axis = 1)
y_defense = defense_feature_set['dfs_points'].tolist()

X_goalies = goalie_feature_set.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season'])
y_goalies = goalie_feature_set['dfs_points'].tolist()


In [0]:
X_train_forwards, X_test_forwards, y_train_forwards, y_test_forwards = train_test_split(X_forwards, y_forwards, test_size=0.2, random_state=42)

X_train_defense, X_test_defense, y_train_defense, y_test_defense = train_test_split(X_defense, y_defense, test_size=0.2, random_state=42)

X_train_goalies, X_test_goalies, y_train_goalies, y_test_goalies = train_test_split(X_goalies, y_goalies, test_size=0.2, random_state=42)

In [16]:
## Do forwards first ## 

train_data_forwards = lgb.Dataset(X_train_forwards, y_train_forwards)

param = {'num_leaves':150,'objective': 'regression','max_depth':7,'learning_rate':.05,'max_bin':200}

num_round = 50

lgbm_forwards = lgb.train(param, train_data_forwards, num_round)

pred_forwards = lgbm_forwards.predict(X_test_forwards)

mean_squared_error(y_test_forwards, pred_forwards)


44.939006861966575

In [17]:
train_data_goalies = lgb.Dataset(X_train_goalies, y_train_goalies)

param = {'num_leaves':150,'objective': 'regression','max_depth':7,'learning_rate':.05,'max_bin':200}

num_round = 50

lgbm_goalies = lgb.train(param, train_data_goalies, num_round)

pred_goalies = lgbm_goalies.predict(X_test_goalies)

mean_squared_error(y_test_goalies, pred_goalies)

111.1453381741206

In [18]:
train_data_defense = lgb.Dataset(X_train_defense, y_train_defense)

param = {'num_leaves':150,'objective': 'regression','max_depth':7,'learning_rate':.05,'max_bin':200}

num_round = 50

lgbm_defense = lgb.train(param, train_data_defense, num_round)

pred_defense = lgbm_defense.predict(X_test_defense)

mean_squared_error(y_test_defense, pred_defense)

29.973142750805287

In [0]:
# Take all the rows from 11/1 and make split them up into goalies, Defense, forwards

november_1_forwards = november_1_skaters.loc[november_1_skaters['primaryPositiontype'] == 'Forward']

X_nov_forwards = november_1_forwards.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season', 'primaryPositioncode', 'primaryPositionname', 'primaryPositiontype'], axis = 1)
y_nov_forwards = november_1_forwards['dfs_points'].tolist()

november_1_defense = november_1_skaters.loc[november_1_skaters['primaryPositiontype'] == 'Defenseman']

X_nov_defense = november_1_defense.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season', 'primaryPositioncode', 'primaryPositionname', 'primaryPositiontype'], axis = 1)
y_nov_defense = november_1_defense['dfs_points'].tolist()

X_nov_goalies = november_1_goalies.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season'], axis = 1)
y_nov_goalies = november_1_goalies['dfs_points'].tolist()

In [20]:
X_nov_goalies.head(10)

,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,goals_against_5,goals_against_10,goals_against_20,goals_against_season,save_percent_1,save_percent_5,save_percent_10,save_percent_20,save_percent_season,shots_against_1,shots_against_5,shots_against_10,shots_against_20,shots_against_season,wins_1,wins_5,wins_10,wins_20,wins_season,shutouts_1,shutouts_5,shutouts_10,shutouts_20,shutouts_season,ot_loss_1,ot_loss_5,ot_loss_10,ot_loss_20,ot_loss_season,es_shots_1,es_shots_5,es_shots_10,es_shots_20,es_shots_season,es_save_percent_1,es_save_percent_5,es_save_percent_10,es_save_percent_20,es_save_percent_season,pp_shots_1,pp_shots_5,pp_shots_10,pp_shots_20,pp_shots_season,pp_save_percent_1,pp_save_percent_5,pp_save_percent_10,pp_save_percent_20,pp_save_percent_season,sh_shots_1,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season
11454,0,14.0,26.4,29.100000,29.450000,29.100000,5.0,2.2,2.500000,2.350000,2.500000,73.684211,91.006761,91.294807,92.205895,91.294807,19.0,28.6,31.600000,31.80,31.600000,0.0,0.400000,0.500000,0.50,0.500000,0.0,0.200000,0.100000,0.150000,0.100000,0.0,0.000000,0.000000,0.050000,0.000000,17.0,24.000000,26.600000,26.450000,26.600000,70.588235,91.905003,92.833119,93.639192,92.833119,2.0,4.200000,4.100000,4.500000,4.100000,100.000000,74.666667,75.833333,79.521605,75.833333,0.0,0.400000,0.900000,0.850000,0.900000,99.019608,100.000000,93.333333,87.878788,93.333333
12361,0,19.0,25.7,26.143478,26.780769,24.111111,0.0,2.0,1.834783,1.819231,2.888889,100.000000,91.714166,92.461940,92.807101,90.176305,19.0,27.7,27.978261,28.60,27.000000,0.0,0.428571,0.517391,0.55,0.111111,0.0,0.178571,0.213043,0.223077,0.000000,0.0,0.185714,0.121739,0.107692,0.222222,15.0,25.007143,25.321739,25.903846,22.777778,100.000000,92.107562,92.861819,93.212621,89.921290,3.0,2.328571,2.326087,2.376923,3.444444,100.000000,85.083475,87.112557,88.062615,80.300454,1.0,0.364286,0.330435,0.319231,0.777778,100.000000,100.000000,100.000000,100.000000,100.000000
12437,1,12.0,22.8,24.000000,25.250000,20.166667,4.0,3.2,2.700000,3.000000,3.000000,75.000000,87.093516,88.595369,88.417193,85.540893,16.0,26.0,26.700000,28.25,23.166667,0.0,0.400000,0.500000,0.40,0.333333,0.0,0.200000,0.100000,0.050000,0.166667,0.0,0.200000,0.100000,0.050000,0.166667,15.0,19.400000,21.300000,23.600000,17.666667,80.000000,90.120370,89.946337,90.037867,88.063272,0.0,6.000000,4.400000,3.900000,5.000000,83.203373,81.666667,84.583333,74.037698,81.666667,1.0,0.600000,1.000000,0.750000,0.500000,0.000000,66.666667,80.000000,88.888889,66.666667
11576,0,41.0,35.4,30.400000,28.400000,35.400000,0.0,2.6,2.900000,3.050000,2.600000,100.000000,92.709919,90.977283,89.902935,92.709919,41.0,38.0,33.300000,31.45,38.000000,1.0,0.600000,0.400000,0.35,0.600000,1.0,0.200000,0.100000,0.050000,0.200000,0.0,0.200000,0.200000,0.150000,0.200000,34.0,30.400000,27.300000,26.500000,30.400000,100.000000,94.288673,92.796351,91.400280,94.288673,7.0,6.400000,5.100000,4.000000,6.400000,100.000000,85.841270,79.920635,74.890873,85.841270,0.0,1.200000,0.900000,0.950000,1.200000,97.368421,100.000000,100.000000,90.909091,100.000000
12704,0,8.0,12.2,18.700000,24.200000,16.285714,1.0,3.0,2.800000,2.800000,2.571429,88.888889,79.253968,85.030671,86.244463,83.798917,9.0,15.2,21.500000,27.00,18.857143,0.0,0.000000,0.200000,0.35,0.285714,0.0,0.000000,0.100000,0.050000,0.142857,0.0,0.200000,0.100000,0.050000,0.142857,9.0,14.200000,18.700000,24.050000,17.000000,88.888889,80.489197,85.553908,85.527316,85.078490,0.0,1.000000,2.100000,2.200000,1.428571,75.023810,62.500000,70.000000,84.117647,58.333333,0.0,0.000000,0.700000,0.750000,0.428571,97.777778,98.253968,100.000000,100.000000,100.000000
6492,0,21.0,28.8,29.500000,29.300000,28.375000,3.0,2.8,2.600000,2.750000,2.625000,87.500000,90.589603,91.319599,90.156012,90.910862,24.0,31.6,32.100000,32.05,31.000000,1.0,0.800000,0.600000,0.50,0.625000,0.0,0.000000,0.000000,0.0000

In [21]:
pred_november_forwards = lgbm_forwards.predict(X_nov_forwards)

mean_squared_error(y_nov_forwards, pred_november_forwards)

48.687075332273125

In [22]:
pred_november_defense = lgbm_defense.predict(X_nov_defense)

mean_squared_error(y_nov_defense, pred_november_defense)

26.535400212036304

In [23]:
pred_november_goalies = lgbm_goalies.predict(X_nov_goalies)

mean_squared_error(y_nov_goalies, pred_november_goalies)

145.96886994834463

In [24]:
len(november_1_skaters)

254

In [25]:
november_1_skaters.head(10)

,player_id,game_id,season,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,assists_5,assists_10,assists_20,assists_season,shots_1,shots_5,shots_10,shots_20,shots_season,hits_1,hits_5,hits_10,hits_20,hits_season,blocks_1,blocks_5,blocks_10,blocks_20,blocks_season,es_toi_1,es_toi_5,es_toi_10,es_toi_20,es_toi_season,pp_toi_1,pp_toi_5,pp_toi_10,pp_toi_20,pp_toi_season,pp_goals_1,pp_goals_5,pp_goals_10,pp_goals_20,pp_goals_season,pp_assists_1,pp_assists_5,pp_assists_10,pp_assists_20,pp_assists_season,takeaways_1,takeaways_5,takeaways_10,takeaways_20,takeaways_season,giveaways_1,giveaways_5,giveaways_10,giveaways_20,giveaways_season,plus_minus_1,plus_minus_5,plus_minus_10,plus_minus_20,plus_minus_season,sh_goals_1,sh_goals_5,sh_goals_10,sh_goals_20,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points,primaryPositioncode,primaryPositionname,primaryPositiontype
1732,8466138,203,2019,1,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.3,0.50,0.307692,0.0,0.6,0.9,0.95,0.846154,1.0,0.4,0.2,0.20,0.307692,0.0,0.4,0.2,0.25,0.230769,787.0,754.2,751.5,754.30,739.384615,57.0,106.8,88.4,138.80,107.846154,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.1,0.25,0.076923,1.0,0.8,0.9,0.90,0.923077,0.0,0.8,0.5,0.55,0.615385,0.0,0.0,-0.2,-0.40,-0.461538,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,7.3,C,Center,Forward
2124,8466139,203,2019,1,0.0,0.0,0.2,0.15,0.222222,0.0,0.6,0.4,0.20,0.444444,0.0,0.8,1.7,1.75,1.333333,3.0,1.2,1.4,1.15,1.555556,0.0,0.4,0.5,0.45,0.555556,814.0,879.6,872.5,825.00,855.888889,30.0,98.4,82.2,88.45,86.333333,0.0,0.0,0.1,0.05,0.111111,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.3,0.35,0.222222,1.0,0.2,0.1,0.25,0.111111,0.0,0.4,0.4,0.05,0.333333,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,11.8,C,Center,Forward
14045,8470047,199,2019,0,0.0,0.2,0.1,0.15,0.076923,1.0,0.2,0.2,0.20,0.230769,1.0,2.0,1.3,1.30,1.076923,1.0,0.4,0.4,0.25,0.307692,0.0,0.4,0.4,0.30,0.307692,887.0,804.2,770.8,718.45,773.769231,82.0,167.6,113.2,109.15,130.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.15,0.000000,0.0,1.0,0.8,0.70,0.846154,1.0,-0.2,-0.4,-0.25,-0.461538,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,6.5,C,Center,Forward
14636,8470110,199,2019,0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.10,0.000000,1.0,0.8,0.9,0.90,0.888889,0.0,0.2,0.1,0.35,0.111111,1.0,1.2,1.0,0.85,1.111111,955.0,889.6,824.8,888.25,804.888889,0.0,1.0,0.5,0.70,0.555556,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.2,0.20,0.222222,1.0,0.4,0.5,0.75,0.555556,-1.0,-0.6,0.0,-0.15,-0.111111,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,3.0,D,Defenseman,Defenseman
15032,8470144,199,2019,0,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.0,0.10,0.000000,1.0,1.6,1.4,1.50,1.400000,1.0,1.0,1.0,1.20,1.000000,2.0,0.6,0.5,0.50,0.500000,700.0,646.4,640.5,694.20,640.500000,11.0,15.0,54.4,77.65,54.400000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.05,0.000000,0.0,0.2,0.2,0.35,0.200000,0.0,0.0,0.1,0.20,0.100000,1.0,0.0,-0.2,-0.25,-0.200000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,4.5,C,Center,Forward
15361,8470151,200,2019,1,0.0,0.0,0.1,0.05,0.076923,0.0,0.2,0.2,0.20,0.153846,1.0,0.8,0.9,0.70,0.769231,0.0,0.4,0.5,0.65,0.692308,1.0,1.8,1.5,1.40,1.461538,1191.0,1119.0,1089.1,1125.20,1104.307692,0.0,2.0,1.0,1.45,1.230769,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.2,0.25,0.153846,0.0,0.4,0.8,0.60,0.846154,0.0,0.0,-0.2,0.00,-0.076923,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,2.8,D,Defenseman,Defenseman
16154,8470187,197,2019,1,0.0,0.0,0.1,0.05,0.100000,1.0,0.8,0.5,0.40,0.500000,3.0,2.0,1.8,1.65,1.800000,0.0,1.6,2.2,2.15,2.200000,3.0,2.0,2.2,2.15,2.200000,888.0,960.0,996.1,993.85,996.100000,0.0,0.0,9.2,6.35,9.200000,0.0,0.0,0.1,0.05,0.100000,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.2,0.20,0.200000,5.0,1.6,1.1,1.10,1.100000,1.0,0.8,0.3,0.20,0.300000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,9.5,D,Defenseman,Defenseman
16665,8470257,200,2019,1,0.0,0.0,0.0,0.10,0.000000,0.0,0.4,0.4,0.25,0.307692,2.0,1.4,1.3,1.65,1.461538,1.0,0.8,0.4,0.35,0.384615,1.0,1.0,1

In [0]:
drop_columns = []

keep_columns = ['player_id', 'game_id', 'season', 'primaryPositionname']

for column in november_1_skaters.columns.values:
  if column not in keep_columns:
    if "_10" not in column:
      drop_columns.append(column)


In [27]:
november_1_skaters.drop(drop_columns, inplace = True, axis = 1)

november_1_skaters.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,player_id,game_id,season,goals_10,assists_10,shots_10,hits_10,blocks_10,es_toi_10,pp_toi_10,pp_goals_10,pp_assists_10,takeaways_10,giveaways_10,plus_minus_10,sh_goals_10,sh_assists_10,primaryPositionname
1732,8466138,203,2019,0.0,0.3,0.9,0.2,0.2,751.5,88.4,0.0,0.1,0.9,0.5,-0.2,0.0,0.0,Center
2124,8466139,203,2019,0.2,0.4,1.7,1.4,0.5,872.5,82.2,0.1,0.0,0.3,0.1,0.4,0.0,0.0,Center
14045,8470047,199,2019,0.1,0.2,1.3,0.4,0.4,770.8,113.2,0.0,0.0,0.0,0.8,-0.4,0.0,0.0,Center
14636,8470110,199,2019,0.0,0.0,0.9,0.1,1.0,824.8,0.5,0.0,0.0,0.2,0.5,0.0,0.0,0.0,Defenseman
15032,8470144,199,2019,0.0,0.0,1.4,1.0,0.5,640.5,54.4,0.0,0.0,0.2,0.1,-0.2,0.0,0.0,Center
15361,8470151,200,2019,0.1,0.2,0.9,0.5,1.5,1089.1,1.0,0.0,0.0,0.2,0.8,-0.2,0.0,0.0,Defenseman
16154,8470187,197,2019,0.1,0.5,1.8,2.2,2.2,996.1,9.2,0.1,0.0,0.2,1.1,0.3,0.0,0.0,Defenseman
16665,8470257,200,2019,0.0,0.4,1.3,0.4,1.4,672.8,4.1,0.0,0.0,0.6,0.3,-0.2,0.0,0.0,Left Wing
21730,8470612,202,2019,0.5,0.2,2.9,1.7,1.1,896.3,128.5,0.2,0.0,0.3,1.8,0.3,0.0,0.0,Center
22122,8470613,203,2019,0.3,0.9,2.4,1.2,1.3,1224.3,135.9,0.1,0.5,0.7,1.4,-0.6,0.0,0.0,Defenseman


In [0]:
players = pd.read_csv('players.csv')


In [0]:
part_1 = november_1_skaters.merge(right = players[['id', 'fullName']], left_on = 'player_id', right_on = 'id', how = 'left')


In [30]:
part_1.head(10)

,player_id,game_id,season,goals_10,assists_10,shots_10,hits_10,blocks_10,es_toi_10,pp_toi_10,pp_goals_10,pp_assists_10,takeaways_10,giveaways_10,plus_minus_10,sh_goals_10,sh_assists_10,primaryPositionname,id,fullName
0,8466138,203,2019,0.0,0.3,0.9,0.2,0.2,751.5,88.4,0.0,0.1,0.9,0.5,-0.2,0.0,0.0,Center,8466138,Joe Thornton
1,8466139,203,2019,0.2,0.4,1.7,1.4,0.5,872.5,82.2,0.1,0.0,0.3,0.1,0.4,0.0,0.0,Center,8466139,Patrick Marleau
2,8470047,199,2019,0.1,0.2,1.3,0.4,0.4,770.8,113.2,0.0,0.0,0.0,0.8,-0.4,0.0,0.0,Center,8470047,Valtteri Filppula
3,8470110,199,2019,0.0,0.0,0.9,0.1,1.0,824.8,0.5,0.0,0.0,0.2,0.5,0.0,0.0,0.0,Defenseman,8470110,Trevor Daley
4,8470144,199,2019,0.0,0.0,1.4,1.0,0.5,640.5,54.4,0.0,0.0,0.2,0.1,-0.2,0.0,0.0,Center,8470144,Frans Nielsen
5,8470151,200,2019,0.1,0.2,0.9,0.5,1.5,1089.1,1.0,0.0,0.0,0.2,0.8,-0.2,0.0,0.0,Defenseman,8470151,Jay Bouwmeester
6,8470187,197,2019,0.1,0.5,1.8,2.2,2.2,996.1,9.2,0.1,0.0,0.2,1.1,0.3,0.0,0.0,Defenseman,8470187,Johnny Boychuk
7,8470257,200,2019,0.0,0.4,1.3,0.4,1.4,672.8,4.1,0.0,0.0,0.6,0.3,-0.2,0.0,0.0,Left Wing,8470257,Alexander Steen
8,8470612,202,2019,0.5,0.2,2.9,1.7,1.1,896.3,128.5,0.2,0.0,0.3,1.8,0.3,0.0,0.0,Center,8470612,Ryan Getzlaf
9,8470613,203,2019,0.3,0.9,2.4,1.2,1.3,1224.3,135.9,0.1,0.5,0.7,1.4,-0.6,0.0,0.0,Defenseman,8470613,Brent Burns


In [0]:
cols = list(part_1.columns.values)
cols.insert(0, cols[-1])

part_1 = part_1[cols[0:-1]]

part_1.head(10)
part_1.drop('id', axis = 1, inplace = True)

In [0]:
part_1 = part_1.sort_values('game_id').reset_index()
part_1.to_csv('10_game_averages.csv')

In [65]:
print(len(part_1))

254


In [67]:
drop_columns = []

keep_columns = ['player_id', 'game_id', 'season']

for column in november_1_goalies.columns.values:
  if column not in keep_columns:
    if "_10" not in column:
      drop_columns.append(column)


november_1_goalies.drop(drop_columns, inplace = True, axis = 1)

part_2 = november_1_goalies.merge(right = players[['id', 'fullName']], left_on = 'player_id', right_on = 'id', how = 'left')

cols = list(part_2.columns.values)
cols.insert(0, cols[-1])

part_2 = part_2[cols[0:-1]]

part_2.head(10)
part_2.drop('id', axis = 1, inplace = True)

part_2 = part_2.sort_values('game_id').reset_index()
part_2.to_csv('10_game_averages_goalies.csv')

print(len(part_2))

17


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
part_2 = part_2.drop(['level_0', 'index'], axis = 1).head()
part_2.to_csv('10_game_averages_goalies.csv')

In [0]:

def f(row):
    if row['timeOnIce'] > 3600 and row['decision'] == 'L':
        val = 1
    else:
        val = 0
    return val

def custom_to_datetime(date):
    # If the time is 24, set it to 0 and increment day by 1
    minutes = date[0:2]
    if ':' in minutes:
        return pd.to_datetime(date, format = '%M:%S')
    else:
        minutes = int(minutes)
        if minutes >= 60:
            newDate = str(minutes - 60) + date[2:]
            return pd.to_datetime(newDate, format = '%M:%S') + pd.Timedelta(hours=1)
        else:
            return pd.to_datetime(date, format = '%M:%S')


p_stats = pd.read_csv('player_stats_by_game.csv').sort_values(by = ['player_id', 'game_id'])

p_stats = p_stats[(p_stats['season'] == 2019) & (p_stats['game_id'] < 204)].reset_index()

goalie_stats = pd.read_csv('goalie_stats_by_game.csv').sort_values(by=['player_id', 'season', 'game_id']).reset_index(drop=True) 

goalie_stats['timeOnIce'] = goalie_stats['timeOnIce'].apply(custom_to_datetime) 
goalie_stats['timeOnIce'] = goalie_stats['timeOnIce'].dt.hour * 3600 + goalie_stats['timeOnIce'].dt.minute * 60 + goalie_stats['timeOnIce'].dt.second
goalie_stats['ot_loss'] = goalie_stats.apply(f, axis = 1)
goalie_stats.head(10)
g_stats = goalie_stats[(goalie_stats['season'] == 2019) & (goalie_stats['game_id'] < 204)].reset_index()


In [103]:
p_stats.head()
print(len(g_stats))

431


In [104]:
part2 = part_1[['player_id', 'fullName']]
part2_goalie = part_2[['player_id', 'fullName']]

print(len(part2_goalie))
agg_cols = {
            'assists' : 'skaterStatsassists', 
            'goals' : 'skaterStatsgoals', 
            'shots' : 'skaterStatsshots', 
            'hits' : 'skaterStatshits', 
            'pp_goals' : 'skaterStatspowerPlayGoals',
            'takeaways' : 'skaterStatstakeaways', 
            'giveaways' : 'skaterStatsgiveaways', 
            'es_toi' : 'skaterStatsevenTimeOnIce', 
            'pp_toi' : 'skaterStatspowerPlayTimeOnIce',
            'blocks' : 'skaterStatsblocked', 
            'pp_assists' : 'skaterStatspowerPlayAssists', 
            'plus_minus' : 'skaterStatsplusMinus', 
            'sh_goals' : 'skaterStatsshortHandedGoals',
            'sh_assists' : 'skaterStatsshortHandedAssists'
           }


agg_cols_goalie = {
    'saves' : 'saves',
    'goals_against' : 'goals_against', 
    'save_percent' : 'savePercentage', 
    'shots_against' : 'shots', 
    'wins' : 'win', 
    'shutouts' : 'shutout', 
    'ot_loss' : 'ot_loss',
    'es_shots' : 'evenShotsAgainst', 
    'es_save_percent' : 'evenStrengthSavePercentage', 
    'pp_shots' : 'powerPlayShotsAgainst', 
    'pp_save_percent' : 'powerPlaySavePercentage', 
    'sh_shots' : 'shortHandedShotsAgainst', 
    'sh_save_percent' : 'shortHandedSavePercentage',
}

17


In [0]:

rows = []

for i in range(0, len(part2)):

  dic = {}

  temp_id = part2['player_id'][i]

  dic['player_id'] = temp_id
  
  temp_game = p_stats[(p_stats['player_id'] == temp_id) & (p_stats['game_id'] >= 196) & (p_stats['game_id'] <= 203)].index


  for j in range(1, 6):
    index = temp_game - j

    dfs_points = (8.5 * p_stats[agg_cols['goals']].iloc[index]) + (5 * p_stats[agg_cols['assists']].iloc[index]) + (1.5 * p_stats[agg_cols['shots']].iloc[index]) + (1.3 * p_stats[agg_cols['blocks']].iloc[index]) + (2 * (p_stats[agg_cols['sh_goals']].iloc[index] + p_stats[agg_cols['sh_assists']].iloc[index]))
        
    bonus_categories = ['goals', 'shots', 'blocks', 'points']
    
    for bonus in bonus_categories:
        if bonus == 'shots':
            if p_stats[agg_cols[bonus]].iloc[i] >= 5:
                dfs_points += 3
        elif bonus == 'points':
            if (p_stats[agg_cols['goals']].iloc[i] + p_stats[agg_cols['assists']].iloc[i]) >= 3:
                dfs_points += 3
        else:
            if p_stats[agg_cols[bonus]].iloc[i] >= 3:
                dfs_points += 3

    dic[str(j) + '_game'] = float(dfs_points)

  rows.append(dic)


In [106]:
part2 = pd.DataFrame(rows)

part2.head()

print(len(part2_goalie))

17


In [0]:
rows_goalie = []

for i in range(0, len(part2_goalie)):

  dic = {}

  temp_id = part2_goalie['player_id'][i]

  dic['player_id'] = temp_id
  
  temp_game = g_stats[(g_stats['player_id'] == temp_id) & (g_stats['game_id'] >= 196) & (g_stats['game_id'] <= 203)].index


  for j in range(1, 6):
    index = temp_game - j

    dfs_points = float((6 * g_stats[agg_cols_goalie['wins']].iloc[index]) + (0.7 * g_stats[agg_cols_goalie['saves']].iloc[index]) + (-3.5 * g_stats[agg_cols_goalie['goals_against']].iloc[index]) + (4 * g_stats[agg_cols_goalie['shutouts']].iloc[index]) + (2 * (g_stats[agg_cols_goalie['ot_loss']].iloc[index])))

    if (int(goalie_stats[agg_cols_goalie['saves']].iloc[index])) >= 35:
         dfs_points += 3

    dic[str(j) + '_game'] = float(dfs_points)

  rows_goalie.append(dic)

In [108]:
goalie_df = pd.DataFrame(rows_goalie)
goalie_df

,player_id,1_game,2_game,3_game,4_game,5_game
0,8478406,-5.6,27.5,19.3,4.8,-4.2
1,8479394,2.1,-11.2,-2.1,-7.0,13.4
2,8471306,23.0,22.8,13.7,10.5,4.4
3,8476883,16.1,24.9,-8.4,22.3,28.6
4,8476999,38.7,18.8,7.2,30.7,19.0
5,8474651,18.1,16.2,17.9,24.2,17.2
6,8473541,-5.0,9.1,17.5,1.4,22.1
7,8475852,22.1,32.4,6.9,-7.7,31.7
8,8470657,24.2,9.1,11.2,4.2,11.2
9,8478007,13.3,15.5,-1.4,19.3,15.6


In [109]:
len(goalie_df)

17

In [110]:
part2.head()

,player_id,1_game,2_game,3_game,4_game,5_game
0,8477355,14.3,5.8,14.3,9.5,5.8
1,8478067,11.5,4.5,3.0,11.3,19.5
2,8474037,4.5,11.0,25.0,17.8,5.8
3,8477948,6.0,8.0,6.0,9.3,13.8
4,8472382,18.4,7.1,9.5,9.7,7.1


In [0]:
part2 = (pd.concat([part2, goalie_df]))
part2 = part2.merge(right = players[['id', 'fullName']], left_on = 'player_id', right_on = 'id', how = 'left')


In [112]:
part2.head()

,player_id,1_game,2_game,3_game,4_game,5_game,id,fullName
0,8477355,14.3,5.8,14.3,9.5,5.8,8477355,Will Butcher
1,8478067,11.5,4.5,3.0,11.3,19.5,8478067,Oskar Lindblom
2,8474037,4.5,11.0,25.0,17.8,5.8,8474037,James van Riemsdyk
3,8477948,6.0,8.0,6.0,9.3,13.8,8477948,Travis Sanheim
4,8472382,18.4,7.1,9.5,9.7,7.1,8472382,Andy Greene


In [113]:
cols = list(part2.columns.values)
cols.insert(0, cols[-1])

part2 = part2[cols[0:-1]]

part2.head(10)

,fullName,player_id,1_game,2_game,3_game,4_game,5_game,id
0,Will Butcher,8477355,14.3,5.8,14.3,9.5,5.8,8477355
1,Oskar Lindblom,8478067,11.5,4.5,3.0,11.3,19.5,8478067
2,James van Riemsdyk,8474037,4.5,11.0,25.0,17.8,5.8,8474037
3,Travis Sanheim,8477948,6.0,8.0,6.0,9.3,13.8,8477948
4,Andy Greene,8472382,18.4,7.1,9.5,9.7,7.1,8472382
5,Wayne Simmonds,8474190,22.5,8.8,9.0,7.3,5.6,8474190
6,Shayne Gostisbehere,8476906,9.1,4.3,19.5,4.3,6.5,8476906
7,Sean Couturier,8476461,12.3,4.5,9.0,10.3,8.0,8476461
8,Jakub Voracek,8474161,6.0,21.0,24.5,3.0,11.0,8474161
9,Claude Giroux,8473512,12.0,18.8,14.0,14.3,10.3,8473512


In [114]:
part2 = part2.sort_values('player_id').reset_index()
part2.head()

,index,fullName,player_id,1_game,2_game,3_game,4_game,5_game,id
0,253,Joe Thornton,8466138,3.0,4.5,5.8,9.3,4.5,8466138
1,228,Patrick Marleau,8466139,0.0,3.0,1.3,9.3,10.0,8466139
2,116,Valtteri Filppula,8470047,6.5,12.8,6.0,2.8,3.0,8470047
3,126,Trevor Daley,8470110,5.8,4.5,8.4,4.3,5.8,8470110
4,117,Frans Nielsen,8470144,4.1,3.0,1.5,4.5,2.8,8470144


In [0]:
part2.drop(['index', 'id'], inplace = True, axis = 1)


In [117]:
part2.head(10)
print(len(part2))

271


In [0]:
part2.to_csv('last_5_games.csv')

In [120]:
prices = pd.read_csv('player_prices.csv')
prices.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68
1,G,Jake Allen (13678960),Jake Allen,13678960,G,8200,CLS@STL 11/01/2019 08:00PM ET,STL,4.75
2,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70
3,LW,Alex Ovechkin (13678140),Alex Ovechkin,13678140,W/UTIL,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,19.41
4,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99


In [131]:
len(prices)

583

In [132]:
len(X_nov_forwards)
len(pred_november_forwards)

X_nov_forwards['pred'] = pred_november_forwards

X_nov_forwards.head()

,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,assists_5,assists_10,assists_20,assists_season,shots_1,shots_5,shots_10,shots_20,shots_season,hits_1,hits_5,hits_10,hits_20,hits_season,blocks_1,blocks_5,blocks_10,blocks_20,blocks_season,es_toi_1,es_toi_5,es_toi_10,es_toi_20,es_toi_season,pp_toi_1,pp_toi_5,pp_toi_10,pp_toi_20,pp_toi_season,pp_goals_1,pp_goals_5,pp_goals_10,pp_goals_20,pp_goals_season,pp_assists_1,pp_assists_5,pp_assists_10,pp_assists_20,pp_assists_season,takeaways_1,takeaways_5,takeaways_10,takeaways_20,takeaways_season,giveaways_1,giveaways_5,giveaways_10,giveaways_20,giveaways_season,plus_minus_1,plus_minus_5,plus_minus_10,plus_minus_20,plus_minus_season,sh_goals_1,sh_goals_5,sh_goals_10,sh_goals_20,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,pred
1732,1,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.3,0.50,0.307692,0.0,0.6,0.9,0.95,0.846154,1.0,0.4,0.2,0.20,0.307692,0.0,0.4,0.2,0.25,0.230769,787.0,754.2,751.5,754.30,739.384615,57.0,106.8,88.4,138.80,107.846154,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.1,0.25,0.076923,1.0,0.8,0.9,0.90,0.923077,0.0,0.8,0.5,0.55,0.615385,0.0,0.0,-0.2,-0.40,-0.461538,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,5.681807
2124,1,0.0,0.0,0.2,0.15,0.222222,0.0,0.6,0.4,0.20,0.444444,0.0,0.8,1.7,1.75,1.333333,3.0,1.2,1.4,1.15,1.555556,0.0,0.4,0.5,0.45,0.555556,814.0,879.6,872.5,825.00,855.888889,30.0,98.4,82.2,88.45,86.333333,0.0,0.0,0.1,0.05,0.111111,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.3,0.35,0.222222,1.0,0.2,0.1,0.25,0.111111,0.0,0.4,0.4,0.05,0.333333,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,6.250026
14045,0,0.0,0.2,0.1,0.15,0.076923,1.0,0.2,0.2,0.20,0.230769,1.0,2.0,1.3,1.30,1.076923,1.0,0.4,0.4,0.25,0.307692,0.0,0.4,0.4,0.30,0.307692,887.0,804.2,770.8,718.45,773.769231,82.0,167.6,113.2,109.15,130.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.15,0.000000,0.0,1.0,0.8,0.70,0.846154,1.0,-0.2,-0.4,-0.25,-0.461538,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,6.663596
15032,0,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.0,0.10,0.000000,1.0,1.6,1.4,1.50,1.400000,1.0,1.0,1.0,1.20,1.000000,2.0,0.6,0.5,0.50,0.500000,700.0,646.4,640.5,694.20,640.500000,11.0,15.0,54.4,77.65,54.400000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.05,0.000000,0.0,0.2,0.2,0.35,0.200000,0.0,0.0,0.1,0.20,0.100000,1.0,0.0,-0.2,-0.25,-0.200000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,4.961054
16665,1,0.0,0.0,0.0,0.10,0.000000,0.0,0.4,0.4,0.25,0.307692,2.0,1.4,1.3,1.65,1.461538,1.0,0.8,0.4,0.35,0.384615,1.0,1.0,1.4,1.10,1.461538,825.0,673.4,672.8,660.00,677.307692,0.0,8.2,4.1,9.75,3.615385,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.6,0.6,0.70,0.538462,0.0,0.4,0.3,0.20,0.307692,0.0,0.2,-0.2,0.15,-0.076923,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,5.644703


In [133]:
len(X_nov_defense)
len(pred_november_defense)

86

In [134]:
prices.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68
1,G,Jake Allen (13678960),Jake Allen,13678960,G,8200,CLS@STL 11/01/2019 08:00PM ET,STL,4.75
2,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70
3,LW,Alex Ovechkin (13678140),Alex Ovechkin,13678140,W/UTIL,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,19.41
4,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99


In [135]:
skater_feature_set = pd.read_csv('feature_matrix_w_position.csv')

## Perform lgb analysis on 3 sets of players (forwards, defensemen, goalies) ##

skater_feature_set = skater_feature_set.drop(columns = ['Unnamed: 0'], axis = 1)

skater_feature_set = skater_feature_set.replace('H', 1)
skater_feature_set = skater_feature_set.replace('A', 0)


after_november_1_skaters = skater_feature_set[(skater_feature_set['season'] == 2019) & (skater_feature_set['game_id'] >= 196)]
november_1_skaters = after_november_1_skaters[(skater_feature_set['game_id'] <= 203)]


skater_feature_set.drop(after_november_1_skaters.index, inplace = True)
print(len(november_1_skaters))
november_1_skaters

288


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


,player_id,game_id,season,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,assists_5,assists_10,assists_20,assists_season,shots_1,shots_5,shots_10,shots_20,shots_season,hits_1,hits_5,hits_10,hits_20,hits_season,blocks_1,blocks_5,blocks_10,blocks_20,blocks_season,es_toi_1,es_toi_5,es_toi_10,es_toi_20,es_toi_season,pp_toi_1,pp_toi_5,pp_toi_10,pp_toi_20,pp_toi_season,pp_goals_1,pp_goals_5,pp_goals_10,pp_goals_20,pp_goals_season,pp_assists_1,pp_assists_5,pp_assists_10,pp_assists_20,pp_assists_season,takeaways_1,takeaways_5,takeaways_10,takeaways_20,takeaways_season,giveaways_1,giveaways_5,giveaways_10,giveaways_20,giveaways_season,plus_minus_1,plus_minus_5,plus_minus_10,plus_minus_20,plus_minus_season,sh_goals_1,sh_goals_5,sh_goals_10,sh_goals_20,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points,primaryPositioncode,primaryPositionname,primaryPositiontype
1732,8466138,203,2019,1,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.3,0.50,0.307692,0.0,0.6,0.9,0.95,0.846154,1.0,0.4,0.2,0.20,0.307692,0.0,0.4,0.2,0.25,0.230769,787.0,754.2,751.5,754.30,739.384615,57.0,106.8,88.4,138.80,107.846154,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.1,0.25,0.076923,1.0,0.8,0.9,0.90,0.923077,0.0,0.8,0.5,0.55,0.615385,0.0,0.0,-0.2,-0.40,-0.461538,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,7.3,C,Center,Forward
2124,8466139,203,2019,1,0.0,0.0,0.2,0.15,0.222222,0.0,0.6,0.4,0.20,0.444444,0.0,0.8,1.7,1.75,1.333333,3.0,1.2,1.4,1.15,1.555556,0.0,0.4,0.5,0.45,0.555556,814.0,879.6,872.5,825.00,855.888889,30.0,98.4,82.2,88.45,86.333333,0.0,0.0,0.1,0.05,0.111111,0.0,0.0,0.0,0.00,0.000000,0.0,0.2,0.3,0.35,0.222222,1.0,0.2,0.1,0.25,0.111111,0.0,0.4,0.4,0.05,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,11.8,C,Center,Forward
14045,8470047,199,2019,0,0.0,0.2,0.1,0.15,0.076923,1.0,0.2,0.2,0.20,0.230769,1.0,2.0,1.3,1.30,1.076923,1.0,0.4,0.4,0.25,0.307692,0.0,0.4,0.4,0.30,0.307692,887.0,804.2,770.8,718.45,773.769231,82.0,167.6,113.2,109.15,130.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.15,0.000000,0.0,1.0,0.8,0.70,0.846154,1.0,-0.2,-0.4,-0.25,-0.461538,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,6.5,C,Center,Forward
14636,8470110,199,2019,0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.10,0.000000,1.0,0.8,0.9,0.90,0.888889,0.0,0.2,0.1,0.35,0.111111,1.0,1.2,1.0,0.85,1.111111,955.0,889.6,824.8,888.25,804.888889,0.0,1.0,0.5,0.70,0.555556,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.2,0.20,0.222222,1.0,0.4,0.5,0.75,0.555556,-1.0,-0.6,0.0,-0.15,-0.111111,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,3.0,D,Defenseman,Defenseman
15032,8470144,199,2019,0,0.0,0.0,0.0,0.05,0.000000,0.0,0.0,0.0,0.10,0.000000,1.0,1.6,1.4,1.50,1.400000,1.0,1.0,1.0,1.20,1.000000,2.0,0.6,0.5,0.50,0.500000,700.0,646.4,640.5,694.20,640.500000,11.0,15.0,54.4,77.65,54.400000,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.05,0.000000,0.0,0.2,0.2,0.35,0.200000,0.0,0.0,0.1,0.20,0.100000,1.0,0.0,-0.2,-0.25,-0.200000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,4.5,C,Center,Forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216036,8480839,198,2019,0,0.0,0.0,0.0,0.10,0.076923,0.0,0.2,0.5,0.55,0.692308,1.0,2.0,1.8,1.80,1.692308,0.0,0.6,0.4,1.00,0.692308,0.0,0.0,0.7,0.65,0.615385,1038.0,943.8,902.4,985.55,911.615385,98.0,237.8,260.2,198.70,241.538462,0.0,0.0,0.0,0.05,0.000000,0.0,0.2,0.4,0.40,0.538462,0.0,0.0,0.0,0.25,0.076923,0.0,1.2,0.8,0.60,0.615385,0.0,-0.8,-0.6,-0.35,-0.153846,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,9.5,D,Defenseman,Defenseman
216877,8481019,203,2019,0,0.0,0.0,NaN,NaN,0.038462,0.0,0.0,NaN,NaN,0.230769,0.0,0.2,NaN,NaN,0.730769,0.0,0.2,NaN,NaN,1.076923,0.0,0.2,NaN,NaN,0.269231,333.0,329.0,NaN,NaN,509.5

In [0]:
november_1_forwards = november_1_skaters.loc[november_1_skaters['primaryPositiontype'] == 'Forward']
forward_player_id = november_1_forwards['player_id']
X_nov_forwards = november_1_forwards.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season', 'primaryPositioncode', 'primaryPositionname', 'primaryPositiontype'], axis = 1)
y_nov_forwards = november_1_forwards['dfs_points'].tolist()

november_1_defense = november_1_skaters.loc[november_1_skaters['primaryPositiontype'] == 'Defenseman']
forward_defense_id = november_1_defense['player_id']
X_nov_defense = november_1_defense.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season', 'primaryPositioncode', 'primaryPositionname', 'primaryPositiontype'], axis = 1)
y_nov_defense = november_1_defense['dfs_points'].tolist()

In [137]:
pred_november_forwards = lgbm_forwards.predict(X_nov_forwards)

mean_squared_error(y_nov_forwards, pred_november_forwards)

47.782155504319235

In [138]:
pred_november_defense = lgbm_defense.predict(X_nov_defense)

mean_squared_error(y_nov_defense, pred_november_defense)

25.66389712557911

In [139]:
goalies = pd.read_csv('goalie_feature_matrix.csv')

goalies = goalies.drop(columns = ['Unnamed: 0'], axis = 1)

goalies = goalies.replace('H', 1)
goalies = goalies.replace('A', 0)

goalies = goalies.fillna(goalies.groupby(['player_id','season']).transform('mean'))
print(len(goalies))
goalies.sort_values('season').tail(10)

after_november_1_goalies = goalies[(goalies['season'] == 2019) & (goalies['game_id'] >= 196)]
november_1_goalies = after_november_1_goalies[(after_november_1_goalies['game_id'] <= 203)]

goalies.drop(after_november_1_goalies.index, inplace = True)
print(len(november_1_goalies))
after_november_1_goalies.head(10)


13003
17


,player_id,game_id,season,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,goals_against_5,goals_against_10,goals_against_20,goals_against_season,save_percent_1,save_percent_5,save_percent_10,save_percent_20,save_percent_season,shots_against_1,shots_against_5,shots_against_10,shots_against_20,shots_against_season,wins_1,wins_5,wins_10,wins_20,wins_season,shutouts_1,shutouts_5,shutouts_10,shutouts_20,shutouts_season,ot_loss_1,ot_loss_5,ot_loss_10,ot_loss_20,ot_loss_season,es_shots_1,es_shots_5,es_shots_10,es_shots_20,es_shots_season,es_save_percent_1,es_save_percent_5,es_save_percent_10,es_save_percent_20,es_save_percent_season,pp_shots_1,pp_shots_5,pp_shots_10,pp_shots_20,pp_shots_season,pp_save_percent_1,pp_save_percent_5,pp_save_percent_10,pp_save_percent_20,pp_save_percent_season,sh_shots_1,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season,dfs_points
391,8467950,206,2019,0,34.0,23.2,24.6,26.45,25.571429,2.0,1.8,2.8,2.85,2.571429,94.444444,93.318969,87.472338,89.092006,91.622393,36.0,25.0,27.4,29.30,28.142857,1.0,0.4,0.3,0.25,0.285714,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.05,0.0,21.0,17.6,21.3,24.15,20.285714,95.238095,92.834169,88.093491,89.058999,92.446484,15.0,6.8,5.3,4.45,6.857143,93.333333,95.809524,87.301587,92.100840,88.435374,0.0,0.6,0.8,0.70,1.000000,89.0625,100.000000,93.75,87.500000,91.666667,1.4
392,8467950,225,2019,0,27.0,24.8,23.1,26.15,25.750000,5.0,2.4,3.1,3.00,2.875000,84.375000,92.098731,86.364384,88.596470,90.716468,32.0,27.2,26.2,29.15,28.625000,0.0,0.2,0.2,0.25,0.250000,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.05,0.0,24.0,18.8,19.7,23.90,20.750000,87.500000,92.556391,87.093491,88.778827,91.828174,6.0,7.8,5.6,4.45,6.750000,66.666667,89.142857,87.301587,90.140056,85.714286,2.0,0.6,0.9,0.80,1.125000,100.0000,100.000000,93.75,89.285714,93.750000,5.6
393,8467950,271,2019,0,28.0,23.8,23.8,26.95,26.000000,4.0,3.0,3.0,3.15,3.000000,87.500000,90.186966,87.037460,88.356086,90.359083,32.0,26.8,26.8,30.10,29.000000,0.0,0.2,0.2,0.25,0.222222,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.05,0.0,25.0,19.0,19.8,24.50,21.222222,88.000000,90.989724,87.976825,88.563442,91.402821,6.0,7.2,6.0,4.75,6.666667,100.000000,89.142857,87.301587,90.687831,87.301587,1.0,0.6,1.0,0.85,1.111111,0.0000,50.000000,75.00,78.125000,75.000000,0.0
394,8467950,281,2019,0,20.0,23.4,25.4,26.85,25.400000,4.0,3.0,3.1,3.20,3.100000,83.333333,89.930556,89.656508,88.122752,89.656508,24.0,26.4,28.5,30.05,28.500000,0.0,0.2,0.2,0.25,0.200000,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.05,0.0,22.0,19.6,21.3,24.55,21.300000,81.818182,90.511255,90.444357,88.368637,90.444357,2.0,6.2,6.2,4.65,6.200000,100.000000,92.000000,88.571429,90.687831,88.571429,0.0,0.6,1.0,0.85,1.000000,89.0625,50.000000,75.00,78.125000,75.000000,13.7
395,8467950,304,2019,0,21.0,26.0,23.8,26.45,25.000000,2.0,3.4,2.8,3.05,3.000000,91.304348,88.191425,89.977419,88.423264,89.806312,23.0,29.4,26.6,29.50,28.000000,1.0,0.4,0.3,0.30,0.272727,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.05,0.0,20.0,22.4,20.1,23.90,21.181818,90.000000,88.511255,90.694357,88.626213,90.403961,3.0,6.4,5.5,4.75,5.909091,100.000000,92.000000,89.571429,90.687831,89.610390,0.0,0.6,1.0,0.85,0.909091,89.0625,50.000000,75.00,78.125000,75.000000,9.1
396,8467950,331,2019,0,28.0,24.8,24.0,26.15,25.250000,3.0,3.6,2.7,3.00,3.000000,90.322581,87.367052,90.343010,88.465709,89.849334,31.0,28.4,26.7,29.15,28.250000,0.0,0.2,0.3,0.30,0.250000,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.0,23.0,22.8,20.2,23.60,21.333333,86.956522,86.854941,89.844555,88.663694,90.116674,7.0,4.8,5.8,4.65,6.000000,100.000000,93.333333,94.571429,90.687831,90.476190,1.0,0.8,0.7,0.90,0.916667,100.0000,66.666667,80.00,80.555556,79.166667,30.0
397,8467950,369,2019,0,35.0,26.4,25.6,26.50,26.000000,1.0,2.8,2.6,2.90,2.846154,97.222222,89.936497,91.017614,88.810691,90.416479,36.0,29.2,28.2,29.40,28.846154,1.0,0.4,0.3,0.35,0.307692,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.0,32.0,24.4,21

In [140]:
goalie_id = november_1_goalies['player_id']
X_nov_goalie = november_1_goalies.drop(columns = ['dfs_points', 'player_id', 'game_id', 'season'], axis = 1)
y_nov_goalie = november_1_goalies['dfs_points'].tolist()

X_nov_goalie.head()

,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,goals_against_5,goals_against_10,goals_against_20,goals_against_season,save_percent_1,save_percent_5,save_percent_10,save_percent_20,save_percent_season,shots_against_1,shots_against_5,shots_against_10,shots_against_20,shots_against_season,wins_1,wins_5,wins_10,wins_20,wins_season,shutouts_1,shutouts_5,shutouts_10,shutouts_20,shutouts_season,ot_loss_1,ot_loss_5,ot_loss_10,ot_loss_20,ot_loss_season,es_shots_1,es_shots_5,es_shots_10,es_shots_20,es_shots_season,es_save_percent_1,es_save_percent_5,es_save_percent_10,es_save_percent_20,es_save_percent_season,pp_shots_1,pp_shots_5,pp_shots_10,pp_shots_20,pp_shots_season,pp_save_percent_1,pp_save_percent_5,pp_save_percent_10,pp_save_percent_20,pp_save_percent_season,sh_shots_1,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season
2256,0,31.0,31.4,32.4,31.30,31.714286,1.0,3.2,3.2,3.25,3.142857,96.875000,90.997222,90.719292,90.502394,91.071204,32.0,34.6,35.6,34.55,34.857143,1.0,0.2,0.3,0.40,0.285714,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.000000,27.0,27.6,29.7,29.60,28.714286,96.296296,92.380952,91.951470,91.795686,91.980183,4.0,6.8,5.6,4.40,5.857143,100.000000,87.478632,71.517094,76.480644,76.770452,1.0,0.2,0.3,0.55,0.285714,100.000000,100.000000,100.000000,78.571429,100.000000
3379,1,30.0,25.4,27.0,24.90,27.000000,2.0,2.2,2.2,1.95,2.000000,93.750000,91.348476,91.719210,91.954306,92.327434,32.0,27.6,29.2,26.85,29.000000,1.0,0.6,0.6,0.55,0.666667,0.0,0.0,0.0,0.15,0.0,0.0,0.0,0.0,0.00,0.000000,24.0,24.2,25.5,23.75,25.833333,95.833333,91.935240,93.242130,92.561552,92.789171,6.0,2.8,2.9,2.50,2.666667,83.333333,86.666667,75.476190,79.197995,88.888889,2.0,0.6,0.8,0.60,0.500000,100.000000,100.000000,100.000000,100.000000,100.000000
3542,0,11.0,21.8,25.1,28.45,21.800000,1.0,2.4,2.5,2.50,2.400000,91.666667,90.534091,91.057458,91.766636,90.534091,12.0,24.2,27.6,30.95,24.200000,1.0,0.4,0.5,0.40,0.400000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.1,0.10,0.000000,11.0,18.6,22.2,25.60,18.600000,100.000000,92.690196,91.740352,92.911740,92.690196,1.0,5.4,5.0,4.55,5.400000,0.000000,72.500000,85.138889,81.714703,72.500000,0.0,0.2,0.4,0.80,0.200000,94.444444,100.000000,100.000000,85.714286,100.000000
5647,0,15.0,28.6,28.2,25.35,27.000000,5.0,3.6,2.8,2.65,3.500000,75.000000,87.488900,89.915783,89.698841,87.301356,20.0,32.2,31.0,28.00,30.500000,0.0,0.2,0.5,0.25,0.333333,0.0,0.0,0.1,0.05,0.0,1.0,0.2,0.1,0.10,0.166667,12.0,25.4,24.8,23.00,24.333333,75.000000,89.503030,90.451774,89.485887,88.620946,6.0,5.0,5.1,4.00,4.666667,66.666667,82.976190,90.238095,90.252976,85.813492,2.0,1.8,1.1,1.00,1.500000,100.000000,91.666667,93.333333,96.666667,91.666667
6492,0,21.0,28.8,29.5,29.30,28.375000,3.0,2.8,2.6,2.75,2.625000,87.500000,90.589603,91.319599,90.156012,90.910862,24.0,31.6,32.1,32.05,31.000000,1.0,0.8,0.6,0.50,0.625000,0.0,0.0,0.0,0.00,0.0,0.0,0.2,0.1,0.15,0.125000,19.0,25.2,26.2,26.80,24.625000,89.473684,93.005367,91.793087,90.881675,91.705645,4.0,5.4,5.0,4.30,5.625000,75.000000,83.333333,92.592593,89.947090,90.476190,1.0,1.0,0.9,0.95,0.750000,100.000000,83.333333,91.666667,87.500000,87.500000


In [141]:
pred_november_goalies = lgbm_goalies.predict(X_nov_goalie)

mean_squared_error(y_nov_goalie, pred_november_goalies)

145.96886994834463

In [142]:
print(len(pred_november_defense))

91


In [143]:
print(len(X_nov_defense))

91


In [144]:
print((pred_november_forwards))

[ 5.68180668  6.25002562  6.6635958   4.96105396  5.64470311  9.74426013
  6.02654481  5.90918202  8.21085051 12.89038472 11.9197925  11.48031532
  5.56748916  8.30214493  3.7678865   4.70374616  5.38302863  6.11868967
  6.88855819  6.91325356  5.02256631  3.39765934  5.41756582  9.5918684
  7.26912818  7.01106239 10.40762558  6.43600676  6.69792447  8.28846686
 10.34075283  7.75775579  4.47571357  6.22500963  9.21827605  9.24913282
  5.60774223  7.3267101   9.23031292  4.62731635 10.97935919  7.21878236
  9.60834929  8.4607856   5.27550495  5.41968032 11.12589642  7.13074585
  6.70744455  6.32382381 10.17565986  9.74150795  8.32705988 12.25804658
  9.31731811 10.78951537  5.77456668  3.75996879  4.54593969  7.88273146
  8.90529197  4.86968108  9.34667902 11.13633883  8.27999873  8.76527215
 11.12152037 13.24380995 11.22716501  7.68324713  6.80677203  4.33282223
  6.68274664  8.11713713  4.21171534  6.53923641  4.91910774  6.84212461
  5.83300144  5.94246206  9.44227574 10.10128192  9.

In [145]:
print((forward_player_id))

1732      8466138
2124      8466139
14045     8470047
15032     8470144
16665     8470257
           ...   
214777    8480205
215330    8480797
215885    8480830
217219    8481559
217411    8481650
Name: player_id, Length: 174, dtype: int64


In [0]:
dic = {"player_id" : forward_player_id, "prediction" : pred_november_forwards}

new_df = pd.DataFrame(dic)

dic1 = {"player_id" : forward_defense_id, "prediction" : pred_november_defense}

new_df_df = pd.DataFrame(dic1)

dic2 = {"player_id" : goalie_id, "prediction" : pred_november_goalies}

goalie_df_final = pd.DataFrame(dic2)

In [147]:
new_df.head()
len(new_df)

174

In [149]:
new_df_df.head()
print(len(goalie_df_final))
len(new_df_df)


17


91

In [0]:
final_df = pd.concat([new_df, new_df_df, goalie_df_final], ignore_index = True).reset_index()

final_df.drop('index', inplace = True, axis = 1)

In [151]:
len(final_df)

282

In [152]:
final_df.head()

,player_id,prediction
0,8466138,5.681807
1,8466139,6.250026
2,8470047,6.663596
3,8470144,4.961054
4,8470257,5.644703


In [0]:
final_df = final_df.merge(right = players[['id', 'fullName']], left_on = 'player_id', right_on = 'id', how = 'left')


In [0]:
final_df = final_df.sort_values('prediction').reset_index().drop(['index', 'id'], axis = 1)

In [155]:
prices.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68
1,G,Jake Allen (13678960),Jake Allen,13678960,G,8200,CLS@STL 11/01/2019 08:00PM ET,STL,4.75
2,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70
3,LW,Alex Ovechkin (13678140),Alex Ovechkin,13678140,W/UTIL,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,19.41
4,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99


In [0]:
prices = prices.merge(right = final_df[['fullName', 'prediction']], left_on = 'Name', right_on = 'fullName', how = 'inner')

In [157]:
prices.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,fullName,prediction
0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68,Jordan Binnington,14.815437
1,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70,Philipp Grubauer,13.329057
2,LW,Alex Ovechkin (13678140),Alex Ovechkin,13678140,W/UTIL,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,19.41,Alex Ovechkin,12.890385
3,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99,Braden Holtby,13.895487
4,G,Petr Mrazek (13678947),Petr Mrazek,13678947,G,8000,DET@CAR 11/01/2019 07:30PM ET,CAR,14.68,Petr Mrazek,12.597150


In [0]:
prices.drop('fullName', axis = 1, inplace = True)

In [159]:
prices.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,prediction
0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68,14.815437
1,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70,13.329057
2,LW,Alex Ovechkin (13678140),Alex Ovechkin,13678140,W/UTIL,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,19.41,12.890385
3,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99,13.895487
4,G,Petr Mrazek (13678947),Petr Mrazek,13678947,G,8000,DET@CAR 11/01/2019 07:30PM ET,CAR,14.68,12.597150


In [0]:
prices['prediction'] = prices['prediction'].round(2)

In [161]:
prices.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,prediction
0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68,14.82
1,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70,13.33
2,LW,Alex Ovechkin (13678140),Alex Ovechkin,13678140,W/UTIL,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,19.41,12.89
3,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99,13.90
4,G,Petr Mrazek (13678947),Petr Mrazek,13678947,G,8000,DET@CAR 11/01/2019 07:30PM ET,CAR,14.68,12.60


In [0]:
prices = prices.rename(columns = {"prediction" : "lgbm_projection"})

In [163]:
prices.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,lgbm_projection
0,G,Jordan Binnington (13678944),Jordan Binnington,13678944,G,8400,CLS@STL 11/01/2019 08:00PM ET,STL,15.68,14.82
1,G,Philipp Grubauer (13678945),Philipp Grubauer,13678945,G,8200,DAL@COL 11/01/2019 09:00PM ET,COL,16.70,13.33
2,LW,Alex Ovechkin (13678140),Alex Ovechkin,13678140,W/UTIL,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,19.41,12.89
3,G,Braden Holtby (13678946),Braden Holtby,13678946,G,8100,BUF@WAS 11/01/2019 07:00PM ET,WAS,10.99,13.90
4,G,Petr Mrazek (13678947),Petr Mrazek,13678947,G,8000,DET@CAR 11/01/2019 07:30PM ET,CAR,14.68,12.60


In [164]:
print(len(prices))

280


In [165]:
print(prices)

    Position                     Name + ID  ... AvgPointsPerGame  lgbm_projection
0          G  Jordan Binnington (13678944)  ...            15.68            14.82
1          G   Philipp Grubauer (13678945)  ...            16.70            13.33
2         LW      Alex Ovechkin (13678140)  ...            19.41            12.89
3          G      Braden Holtby (13678946)  ...            10.99            13.90
4          G        Petr Mrazek (13678947)  ...            14.68            12.60
..       ...                           ...  ...              ...              ...
275        D           Tim Heed (13678798)  ...             1.62             3.64
276        D      Mario Ferraro (13678810)  ...             2.54             4.64
277        C      Casey Cizikas (13678008)  ...             3.63             5.77
278        C    Cedric Paquette (13678016)  ...             4.60             4.66
279       LW      Ross Johnston (13678366)  ...             3.95             3.68

[280 rows x 10 

In [0]:
prices.to_csv('player_prices_w_pred.csv')